In [35]:
from svinterface.core.zerod.lpn import LPN

In [86]:
lpn = LPN.from_file("../../data/diseased/AS1_SU0308_stent/results/AS1_SU0308_nonlinear/parameterization/AS1_SU0308.in")
lpn2 = LPN.from_file("../../data/diseased/AS1_SU0308_stent/results/AS1_SU0308_nonlinear/parameterization/LPA_limited_2std/AS1_SU0308.in")
lpn3 = LPN.from_file("../../data/diseased/AS1_SU0308_stent/results/AS1_SU0308_nonlinear/parameterization/RPA_stent_limited_2std/AS1_SU0308.in")
lpn4 = LPN.from_file("../../data/diseased/AS1_SU0308_stent/results/AS1_SU0308_nonlinear/parameterization/RPA_stent_2_limited_2std/AS1_SU0308.in")

In [87]:
tree1 = lpn.get_tree()
tree2 = lpn4.get_tree()

def compare_node_resistances(n1: LPN.Node, n2: LPN.Node):
    assert type(n1) == type(n2), "Node types are not equal"
    if type(n1) == LPN.BranchNode:
        assert len(n1) == len(n2), "Nodes do not contain same number of vessels"
        dR = []
        for idx in range(len(n1)):
            assert n1.ids[idx] == n2.ids[idx], "Nodes ids do not match"
            if n1.vessel_info[idx]['zero_d_element_values']['R_poiseuille'] != 0:
                dR.append((n2.vessel_info[idx]['zero_d_element_values']['R_poiseuille'] - n1.vessel_info[idx]['zero_d_element_values']['R_poiseuille'])/n1.vessel_info[idx]['zero_d_element_values']['R_poiseuille'])
            else:
                dR.append(("NaN", (n2.vessel_info[idx]['zero_d_element_values']['R_poiseuille'] - n1.vessel_info[idx]['zero_d_element_values']['R_poiseuille'])))
        return dR
    elif type(n1) == LPN.JunctionNode:
        assert len(n1.vessel_info[0]['junction_values']['R_poiseuille']) == len(n2.vessel_info[0]['junction_values']['R_poiseuille']), "Nodes do not contain same number of Junctions(1)"
        dR = []
        for idx, (r1, r2) in enumerate(zip(n1.vessel_info[0]['junction_values']['R_poiseuille'], n2.vessel_info[0]['junction_values']['R_poiseuille'])):
            if r1 != 0:
                dR.append((r2 - r1) / r1)
            else:
                dR.append(("NaN", r2 - r1))
        return dR

with open("lol3.txt", "w") as f:
    for idx, (node1, node2) in enumerate(zip(lpn.tree_bfs_iterator(tree1), lpn2.tree_bfs_iterator(tree2))):
        for i in compare_node_resistances(node1, node2):
            f.write(str(i))
            f.write(" ")
        f.write("\n")
            
        
        

In [88]:
def compare_node_resistances(n1: LPN.Node, n2: LPN.Node, n3: LPN.Node, n4: LPN.Node):
    assert type(n1) == type(n2), "Node types are not equal"
    if type(n1) == LPN.BranchNode:
        assert len(n1) == len(n2), "Nodes do not contain same number of vessels"
        dR = []
        for idx in range(len(n1)):
            assert n1.ids[idx] == n2.ids[idx], "Nodes ids do not match"
            r1 = n1.vessel_info[idx]['zero_d_element_values']['R_poiseuille']
            r2 = n2.vessel_info[idx]['zero_d_element_values']['R_poiseuille']
            r3 = n3.vessel_info[idx]['zero_d_element_values']['R_poiseuille']
            r4 = n4.vessel_info[idx]['zero_d_element_values']['R_poiseuille']
            if  r1 + (r2 - r1) + (r3 - r1) + (r4 - r1) < 0:
                dR.append((r1, r2 - r1, r3 - r1, r4 - r1, r1 + (r2 - r1) + (r3 - r1) + (r4 - r1), n1.vessel_info[idx]['vessel_id']))
        return dR
    elif type(n1) == LPN.JunctionNode:
        assert len(n1.vessel_info[0]['junction_values']['R_poiseuille']) == len(n2.vessel_info[0]['junction_values']['R_poiseuille']), "Nodes do not contain same number of Junctions(1)"
        dR = []
        for idx, (r1, r2, r3, r4) in enumerate(zip(n1.vessel_info[0]['junction_values']['R_poiseuille'], n2.vessel_info[0]['junction_values']['R_poiseuille'], n3.vessel_info[0]['junction_values']['R_poiseuille'], n4.vessel_info[0]['junction_values']['R_poiseuille'])):
            if r1 + (r2 - r1) + (r3 - r1) + (r4 - r1) < 0:
                dR.append((r1, r2 - r1, r3 - r1, r4 - r1, r1 + (r2 - r1) + (r3 - r1) + (r4 - r1), n1.vessel_info[0]['junction_name']))
        return dR

In [89]:
tree1 = lpn.get_tree()
tree2 = lpn2.get_tree()
tree3 = lpn3.get_tree()
tree4 = lpn4.get_tree()

In [90]:
for idx, (node1, node2, node3, node4) in enumerate(zip(lpn.tree_bfs_iterator(tree1), lpn2.tree_bfs_iterator(tree2), lpn.tree_bfs_iterator(tree3), lpn.tree_bfs_iterator(tree4))):
    tmp = compare_node_resistances(node1,node2,node3, node4)
    if len(tmp) > 0:
        print(tmp)